# Modeling Body Fat Percentage
### Performance Metrics
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- Mean Absolute Error (MAE)

### Machine Learning Algorithms
- Linear Regression
- Ridge Regression
- Lasso Regression
- Decision Trees
- Random Forest
- K-Nearest Neighbors
- Support Vector Machines

# Import and Prepare Data

In [1]:
# Import packages
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

In [2]:
# Import data
data = []

for i in range(1, 6):
    data_i = pd.read_csv("./../data/processed/M" + str(i) + "_data.csv").set_index('id')
    data.append(data_i)

predictors = pd.read_csv('./../data/processed/predictors.csv').set_index('id')

# Baselines: Calculating Body Fat Percentage from Simple Equations using BMI
There have been a number of studies that attempt to define simple equations to convert BMI to estimated body fat percentage using sex and age. We will generate these estimates as a baseline to our more complicated algorithms to evaluate performance.

## Baseline 1: 1991 Study

This algorithm is based on [Deurenberg, P et al.](https://www.cambridge.org/core/journals/british-journal-of-nutrition/article/body-mass-index-as-a-measure-of-body-fatness-age-and-sexspecific-prediction-formulas/9C03B18E1A0E4CDB0441644EE64D9AA2) (see `~/references/DeurenbergEtAl_1991.pdf` for the full article).

For children (age 15 and younger), body fat percentage is estimated by: $$1.51 \cdot \text{BMI} - 0.70 \cdot \text{age} - 3.6 \cdot \text{sex} + 1.4$$  
  
For adults (older than age 15), the body fat percentage is estimated by: $$1.20 \cdot \text{BMI} + 0.23 \cdot \text{age} - 10.8 \cdot \text{sex} - 5.4$$  
  
In each of these formulae, the `sex` variable is defined as 1 for males and 0 for females. We calculate these estimates below, under `baseline1`.

In [3]:
def bf_1991(df):
    # Define a few columns for easier reference
    df['adult'] = df['age_yrs'] > 15
    df['sex_opp'] = df['sex'].replace({0:1, 1:0})
    df['bmi'] = df['weight_kg'] / (df['height_cm'] / 100) ** 2

    # Construct estimate column
    # Fill with invalid values (-99)
    df['baseline1'] = -99

    # Calculate estimates for adults
    df['baseline1'].mask(
        df['adult'],
        (1.2 * df['bmi']) + (0.23 * df['age_yrs']) - (10.8 * df['sex_opp']) - 5.4,
        inplace=True
    )

    # Calculate estimates for children
    df['baseline1'].where(
        df['adult'],
        (1.51 * df['bmi']) - (0.7 * df['age_yrs']) - (3.6 * df['sex_opp']) + 1.4,
        inplace=True
    )

    # Return dataframe with new 'baseline1' column
    return df.drop(['adult', 'sex_opp', 'bmi'], axis=1)

bf_1991(data[0]).head()

,weight_kg,height_cm,sex,age_yrs,race,waist_circum_cm,smoker_status,years,M,total_fat_pct,baseline1
id,,,,,,,,,,,
2,75.4,174.0,0,77.0,3,98.0,0.0,1999-2000,1,31.0,31.395057
5,92.5,178.3,0,49.0,3,99.9,2.0,1999-2000,1,31.2,29.985663
6,59.2,162.0,1,19.0,5,81.6,NaN,1999-2000,1,41.5,26.039044
7,78.0,162.9,1,59.0,4,90.7,2.0,1999-2000,1,39.9,43.442292
8,40.7,162.0,0,13.0,3,64.1,NaN,1999-2000,1,19.5,12.117543


Run the calculations on each dataset.

In [4]:
for data_i in data:
    bf_1991(data_i).head()

## Baseline 2: 2002 Study

This algorithm is based on [Jackson, A et al.](https://www.nature.com/articles/0802006) (see `~/references/JacksonEtAl_2002.pdf` for the full article).

This study uses a more general formula, defined by: $$1.39 \cdot \text{BMI} + 0.16 \cdot \text{age} - 10.34 \cdot \text{sex} - 9$$
  
As in the first formulae, the `sex` variable is defined where male is 1 and female is 0. We calculate these estimates below, under `baseline2`.

In [5]:
def bf_2002(df):
    # Define a few columns for easier reference
    df['sex_opp'] = df['sex'].replace({0:1, 1:0})
    df['bmi'] = df['weight_kg'] / (df['height_cm'] / 100) ** 2

    # Construct estimate column
    df['baseline2'] = (1.39 * df['bmi']) + (0.16 * df['age_yrs']) - (10.34 * df['sex_opp']) - 9

    # Return dataframe with new 'baseline2' column
    return df.drop(['sex_opp', 'bmi'], axis=1)

bf_2002(data[0]).head()

,weight_kg,height_cm,sex,age_yrs,race,waist_circum_cm,smoker_status,years,M,total_fat_pct,adult,baseline1,baseline2
id,,,,,,,,,,,,,
2,75.4,174.0,0,77.0,3,98.0,0.0,1999-2000,1,31.0,True,31.395057,27.596858
5,92.5,178.3,0,49.0,3,99.9,2.0,1999-2000,1,31.2,True,29.985663,28.943976
6,59.2,162.0,1,19.0,5,81.6,NaN,1999-2000,1,41.5,True,26.039044,25.394976
7,78.0,162.9,1,59.0,4,90.7,2.0,1999-2000,1,39.9,True,43.442292,41.297071
8,40.7,162.0,0,13.0,3,64.1,NaN,1999-2000,1,19.5,False,12.117543,4.296546


Run the calculations on each dataset.

In [6]:
for data_i in data:
    bf_2002(data_i)

# Algorithm 1: Weight, Height, Sex, Age, and Waist Circumference

Start by loading only the relevant datasets into a list of 5 (one for each imputation) dataframes called `data1`.

In [7]:
data1 = []
predictors_cols1 = ['weight_kg', 'height_cm', 'sex', 'age_yrs', 'waist_circum_cm']

for data_i in data:
    data1.append(data_i.copy()[['weight_kg', 'height_cm', 'sex', 'age_yrs', 'waist_circum_cm', 'total_fat_pct']])

Split the data into training sets and testing sets.

In [8]:
# Fill lists with NaN values so they can be indexed
X_train1 = np.repeat(np.nan, 5).tolist()
X_test1 = np.repeat(np.nan, 5).tolist()
y_train1 = np.repeat(np.nan, 5).tolist()
y_test1 = np.repeat(np.nan, 5).tolist()

# Split by same ids for all imputations
for i, data_i in enumerate(data):
    X_train1[i], X_test1[i], y_train1[i], y_test1[i] = train_test_split(
        data_i[predictors_cols1],
        data_i['total_fat_pct'],
        test_size=0.3,
        random_state=42
    )

## Baselines

Record the performance of the baseline equations.

In [9]:
# Save indices of train set and test set
train_ind_1 = X_train1[0].index.tolist()
test_ind_1 = X_test1[0].index.tolist()

# Create lists to save residuals and metrics
train_resid_bl1 = []
test_resid_bl1 = []

train_resid_bl2 = []
test_resid_bl2 = []

mse_train_bl1 = []
mse_test_bl1 = []
rmse_train_bl1 = []
rmse_test_bl1 = []
mae_train_bl1 = []
mae_test_bl1 = []

mse_train_bl2 = []
mse_test_bl2 = []
rmse_train_bl2 = []
rmse_test_bl2 = []
mae_train_bl2 = []
mae_test_bl2 = []

for i, data_i in enumerate(data):
    # Redefine split data
    train_i = data_i.loc[train_ind_1].copy()
    test_i = data_i.loc[test_ind_1].copy()

    # Save residuals
    train_resid_bl1.append(
        train_i['total_fat_pct'] - train_i['baseline1']
    )
    test_resid_bl1.append(
        test_i['total_fat_pct'] - test_i['baseline1']
    )
    train_resid_bl2.append(
        train_i['total_fat_pct'] - train_i['baseline2']
    )
    test_resid_bl2.append(
        test_i['total_fat_pct'] - test_i['baseline2']
    )

    # Save performance scores
    # MSE for train data (baseline 1)
    mse_train_bl1_i = np.sum((train_resid_bl1[i])**2) / len(train_resid_bl1[i])
    mse_train_bl1.append(mse_train_bl1_i)

    # MSE for test data (baseline 1)
    mse_test_bl1_i = np.sum((test_resid_bl1[i])**2) / len(test_resid_bl1[i])
    mse_test_bl1.append(mse_test_bl1_i)

    # RMSE for train data (baseline 1)
    rmse_train_bl1_i = np.sqrt(mse_train_bl1_i)
    rmse_train_bl1.append(rmse_train_bl1_i)

    # RMSE for test data (baseline 1)
    rmse_test_bl1_i = np.sqrt(mse_test_bl1_i)
    rmse_test_bl1.append(rmse_test_bl1_i)

    # MAE for train data (baseline 1)
    mae_train_bl1_i = np.sum(np.abs(train_resid_bl1[i])) / len(train_resid_bl1[i])
    mae_train_bl1.append(mae_train_bl1_i)

    # MAE for test data (baseline 1)
    mae_test_bl1_i = np.sum(np.abs(test_resid_bl1[i])) / len(test_resid_bl1[i])
    mae_test_bl1.append(mae_test_bl1_i)

    # MSE for train data (baseline 2)
    mse_train_bl2_i = np.sum((train_resid_bl2[i])**2) / len(train_resid_bl2[i])
    mse_train_bl2.append(mse_train_bl2_i)

    # MSE for test data (baseline 2)
    mse_test_bl2_i = np.sum((test_resid_bl2[i])**2) / len(test_resid_bl2[i])
    mse_test_bl2.append(mse_test_bl2_i)

    # RMSE for train data (baseline 2)
    rmse_train_bl2_i = np.sqrt(mse_train_bl2_i)
    rmse_train_bl2.append(rmse_train_bl2_i)

    # RMSE for test data (baseline 2)
    rmse_test_bl2_i = np.sqrt(mse_test_bl2_i)
    rmse_test_bl2.append(rmse_test_bl2_i)

    # MAE for train data (baseline 2)
    mae_train_bl2_i = np.sum(np.abs(train_resid_bl2[i])) / len(train_resid_bl2[i])
    mae_train_bl2.append(mae_train_bl2_i)

    # MAE for test data (baseline 2)
    mae_test_bl2_i = np.sum(np.abs(test_resid_bl2[i])) / len(test_resid_bl2[i])
    mae_test_bl2.append(mae_test_bl2_i)
    

In [10]:
# Organize metrics into dataframe
bl1_results = pd.DataFrame(
    [mse_train_bl1, mse_test_bl1, rmse_train_bl1, rmse_test_bl1, mae_train_bl1, mae_test_bl1],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

bl1_results

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,40.362904,39.819925,6.353181,6.310303,5.079836,5.042850
M2,40.250213,39.910735,6.344306,6.317494,5.076825,5.065291
M3,40.394123,39.964842,6.355637,6.321775,5.086365,5.062050
M4,40.230780,39.936807,6.342774,6.319557,5.073808,5.062003
M5,40.350474,39.541146,6.352202,6.288175,5.084892,5.038733


In [11]:
bl2_results = pd.DataFrame(
    [mse_train_bl2, mse_test_bl2, rmse_train_bl2, rmse_test_bl2, mae_train_bl2, mae_test_bl2],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

bl2_results

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,74.655154,74.833199,8.640321,8.650618,6.617873,6.675169
M2,74.548713,74.811470,8.634160,8.649362,6.618549,6.684939
M3,74.650022,74.961082,8.640024,8.658007,6.625581,6.681407
M4,74.479894,74.820695,8.630173,8.649896,6.615607,6.683822
M5,74.535289,74.389833,8.633382,8.624954,6.620778,6.670983


In [12]:
bl1_results_avgs = pd.DataFrame(bl1_results.mean(axis=0), columns=['BL1']).T
bl1_results_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
BL1,40.317699,39.834691,6.34962,6.311461,5.080345,5.054185


In [13]:
bl2_results_avgs = pd.DataFrame(bl2_results.mean(axis=0), columns=['BL2']).T
bl2_results_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
BL2,74.573814,74.763256,8.635612,8.646567,6.619678,6.679264


## Linear Regression

Train the model (one for each imputation).

In [14]:
# Initialize
model1_lm = np.repeat(LinearRegression(), 5).tolist()

# Train
for i, model_i in enumerate(model1_lm):
    model_i.fit(X_train1[i], y_train1[i])

In [15]:
# Generate test predictions and calculate residuals
y_test_pred1_lm = []
y_train_pred1_lm = []
test_resid_1_lm = []
train_resid_1_lm = []

for i, model_i in enumerate(model1_lm):
    # Train
    y_train_pred1_lm.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_lm.append(
        y_train1[i] - y_train_pred1_lm[i]
    )


    # Test
    y_test_pred1_lm.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_lm.append(
        y_test1[i] - y_test_pred1_lm[i]
    )


Calculate performance metrics (MSE, RMSE, MAE).

In [16]:
mse_train1_lm = []
mse_test1_lm = []
rmse_train1_lm = []
rmse_test1_lm = []
mae_train1_lm = []
mae_test1_lm = []

for i, model_i in enumerate(model1_lm):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_lm[i])**2) / len(train_resid_1_lm[i])
    mse_train1_lm.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_lm[i])**2) / len(test_resid_1_lm[i])
    mse_test1_lm.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_lm.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_lm.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_lm[i])) / len(train_resid_1_lm[i])
    mae_train1_lm.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_lm[i])) / len(test_resid_1_lm[i])
    mae_test1_lm.append(mae_test_i)

In [17]:
# Organize metrics into dataframe
alg1_results_lm = pd.DataFrame(
    [mse_train1_lm, mse_test1_lm, rmse_train1_lm, rmse_test1_lm, mae_train1_lm, mae_test1_lm],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_lm

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,17.175034,16.954400,4.144277,4.117572,3.244504,3.241387
M2,17.168024,17.166120,4.143431,4.143202,3.242117,3.261129
M3,17.084709,17.131540,4.133365,4.139026,3.236623,3.256213
M4,17.072518,17.105916,4.131890,4.135930,3.233129,3.256117
M5,17.133025,16.985049,4.139206,4.121292,3.244010,3.244988


In [18]:
alg1_results_lm_avgs = pd.DataFrame(alg1_results_lm.mean(axis=0), columns=['LM']).T
alg1_results_lm_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
LM,17.126662,17.068605,4.138434,4.131404,3.240077,3.251967


## Ridge Regression

Train the model (one for each imputation)

In [19]:
# Initialize
model1_ridge = np.repeat(Ridge(alpha=1), 5).tolist() # Consider using cross validation to find the optimal alpha (lambda)

# Train
for i, model_i in enumerate(model1_ridge):
    model_i.fit(X_train1[i], y_train1[i])

In [20]:
# Generate test predictions and calculate residuals
y_test_pred1_ridge = []
y_train_pred1_ridge = []
test_resid_1_ridge = []
train_resid_1_ridge = []

for i, model_i in enumerate(model1_ridge):
    # Train
    y_train_pred1_ridge.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_ridge.append(
        y_train1[i] - y_train_pred1_ridge[i]
    )


    # Test
    y_test_pred1_ridge.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_ridge.append(
        y_test1[i] - y_test_pred1_ridge[i]
    )


Calculate performance metrics (MSE, RMSE, MAE).

In [21]:
mse_train1_ridge = []
mse_test1_ridge = []
rmse_train1_ridge = []
rmse_test1_ridge = []
mae_train1_ridge = []
mae_test1_ridge = []

for i, model_i in enumerate(model1_ridge):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_ridge[i])**2) / len(train_resid_1_ridge[i])
    mse_train1_ridge.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_ridge[i])**2) / len(test_resid_1_ridge[i])
    mse_test1_ridge.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_ridge.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_ridge.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_ridge[i])) / len(train_resid_1_ridge[i])
    mae_train1_ridge.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_ridge[i])) / len(test_resid_1_ridge[i])
    mae_test1_ridge.append(mae_test_i)

In [22]:
# Organize metrics into dataframe
alg1_results_ridge = pd.DataFrame(
    [mse_train1_ridge, mse_test1_ridge, rmse_train1_ridge, rmse_test1_ridge, mae_train1_ridge, mae_test1_ridge],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_ridge

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,17.175056,16.954197,4.144280,4.117547,3.244561,3.241432
M2,17.168049,17.165949,4.143434,4.143181,3.242171,3.261180
M3,17.084712,17.131357,4.133366,4.139004,3.236673,3.256260
M4,17.072521,17.105736,4.131891,4.135908,3.233182,3.256167
M5,17.133027,16.984910,4.139206,4.121275,3.244063,3.245043


In [24]:
alg1_results_ridge_avgs = pd.DataFrame(alg1_results_ridge.mean(axis=0), columns=['Ridge_Avg']).T
alg1_results_ridge_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
Ridge_Avg,17.126673,17.06843,4.138435,4.131383,3.24013,3.252016


## Ridge Regression with Cross Validation

Train the model (one for each imputation)

In [25]:
# Initialize
model1_ridge_CV = np.repeat(RidgeCV(alphas=[0.1, 0.2, 0.3, 0.4, 0.5, 1]), 5).tolist()

# Train
for i, model_i in enumerate(model1_ridge_CV):
    model_i.fit(X_train1[i], y_train1[i])

    # Print chosen alpha
    print("Imputation", str(i+1), "alpha:", model_i.alpha_)

Imputation 1 alpha: 0.3
Imputation 2 alpha: 0.3
Imputation 3 alpha: 0.3
Imputation 4 alpha: 0.3
Imputation 5 alpha: 0.3


In [26]:
# Generate test predictions and calculate residuals
y_test_pred1_ridge_CV = []
y_train_pred1_ridge_CV = []
test_resid_1_ridge_CV = []
train_resid_1_ridge_CV = []

for i, model_i in enumerate(model1_ridge_CV):
    # Train
    y_train_pred1_ridge_CV.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_ridge_CV.append(
        y_train1[i] - y_train_pred1_ridge_CV[i]
    )


    # Test
    y_test_pred1_ridge_CV.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_ridge_CV.append(
        y_test1[i] - y_test_pred1_ridge[i]
    )


In [27]:
mse_train1_ridge_CV = []
mse_test1_ridge_CV = []
rmse_train1_ridge_CV = []
rmse_test1_ridge_CV = []
mae_train1_ridge_CV = []
mae_test1_ridge_CV = []

for i, model_i in enumerate(model1_ridge_CV):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_ridge_CV[i])**2) / len(train_resid_1_ridge_CV[i])
    mse_train1_ridge_CV.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_ridge_CV[i])**2) / len(test_resid_1_ridge_CV[i])
    mse_test1_ridge_CV.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_ridge_CV.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_ridge_CV.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_ridge_CV[i])) / len(train_resid_1_ridge_CV[i])
    mae_train1_ridge_CV.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_ridge_CV[i])) / len(test_resid_1_ridge_CV[i])
    mae_test1_ridge_CV.append(mae_test_i)

In [28]:
# Organize metrics into dataframe
alg1_results_ridge_CV = pd.DataFrame(
    [mse_train1_ridge_CV, mse_test1_ridge_CV, rmse_train1_ridge_CV, rmse_test1_ridge_CV, mae_train1_ridge_CV, mae_test1_ridge_CV],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_ridge_CV

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,17.175041,16.954197,4.144278,4.117547,3.244521,3.241432
M2,17.168031,17.165949,4.143432,4.143181,3.242133,3.261180
M3,17.084710,17.131357,4.133365,4.139004,3.236638,3.256260
M4,17.072519,17.105736,4.131890,4.135908,3.233145,3.256167
M5,17.133025,16.984910,4.139206,4.121275,3.244026,3.245043


In [29]:
alg1_results_ridge_avgs_CV = pd.DataFrame(alg1_results_ridge_CV.mean(axis=0), columns=['Ridge_CV_Avg']).T
alg1_results_ridge_avgs_CV

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
Ridge_CV_Avg,17.126665,17.06843,4.138434,4.131383,3.240093,3.252016


## Lasso Regression

Train the model (one for each imputation)

In [30]:
# Initialize
model1_lasso = np.repeat(Lasso(alpha=1), 5).tolist() # Consider using cross validation to find the optimal alpha (lambda)

# Train
for i, model_i in enumerate(model1_lasso):
    model_i.fit(X_train1[i], y_train1[i])

In [31]:
# Generate test predictions and calculate residuals
y_test_pred1_lasso = []
y_train_pred1_lasso = []
test_resid_1_lasso = []
train_resid_1_lasso = []

for i, model_i in enumerate(model1_lasso):
    # Train
    y_train_pred1_lasso.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_lasso.append(
        y_train1[i] - y_train_pred1_lasso[i]
    )


    # Test
    y_test_pred1_lasso.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_lasso.append(
        y_test1[i] - y_test_pred1_lasso[i]
    )


Calculate performance metrics (MSE, RMSE, MAE).

In [32]:
mse_train1_lasso = []
mse_test1_lasso = []
rmse_train1_lasso = []
rmse_test1_lasso = []
mae_train1_lasso = []
mae_test1_lasso = []

for i, model_i in enumerate(model1_lasso):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_lasso[i])**2) / len(train_resid_1_lasso[i])
    mse_train1_lasso.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_lasso[i])**2) / len(test_resid_1_lasso[i])
    mse_test1_lasso.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_lasso.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_lasso.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_lasso[i])) / len(train_resid_1_lasso[i])
    mae_train1_lasso.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_lasso[i])) / len(test_resid_1_lasso[i])
    mae_test1_lasso.append(mae_test_i)

In [33]:
# Organize metrics into dataframe
alg1_results_lasso = pd.DataFrame(
    [mse_train1_lasso, mse_test1_lasso, rmse_train1_lasso, rmse_test1_lasso, mae_train1_lasso, mae_test1_lasso],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_lasso

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,22.020436,21.383061,4.692594,4.624182,3.787748,3.751611
M2,22.017221,21.652731,4.692251,4.653250,3.785483,3.774623
M3,21.888230,21.599266,4.678486,4.647501,3.773179,3.766657
M4,21.876977,21.583036,4.677283,4.645755,3.775101,3.766758
M5,21.935220,21.539103,4.683505,4.641024,3.786183,3.764667


In [34]:
alg1_results_lasso_avgs = pd.DataFrame(alg1_results_lasso.mean(axis=0), columns=['Lasso_Avg']).T
alg1_results_lasso_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
Lasso_Avg,21.947617,21.551439,4.684824,4.642342,3.781539,3.764863


## Lasso Regression with Cross Validation

Train the model (one for each imputation)

In [35]:
# Initialize
model1_lasso_CV = np.repeat(LassoCV(alphas=[0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 1]), 5).tolist() # Optimal alpha is 0

# Train
for i, model_i in enumerate(model1_lasso_CV):
    model_i.fit(X_train1[i], y_train1[i])

    # Print chosen alpha
    print("Imputation", str(i+1), "alpha:", model_i.alpha_)

Imputation 1 alpha: 0.001
Imputation 2 alpha: 0.001
Imputation 3 alpha: 0.001
Imputation 4 alpha: 0.001
Imputation 5 alpha: 0.001


After playing around with the code in the chunk above, it's clear that the optimal alpha value (based on cross validation) is 0. This means that Lasso Regression is not expected to perform any better than Linear Regression, so we will move onto the next algorithm.

## Decision Trees

Train the model (one for each imputation)

In [36]:
# Initialize
model1_dt = np.repeat(DecisionTreeRegressor(max_depth=9, random_state=42), 5).tolist() # The best tree depth seems to be 9

# Train
for i, model_i in enumerate(model1_dt):
    model_i.fit(X_train1[i], y_train1[i])
    print("Tree depth:", str(model_i.get_depth()))

Tree depth: 9
Tree depth: 9
Tree depth: 9
Tree depth: 9
Tree depth: 9


In [37]:
# Generate test predictions and calculate residuals
y_test_pred1_dt = []
y_train_pred1_dt = []
test_resid_1_dt = []
train_resid_1_dt = []

for i, model_i in enumerate(model1_dt):
    # Train
    y_train_pred1_dt.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_dt.append(
        y_train1[i] - y_train_pred1_dt[i]
    )


    # Test
    y_test_pred1_dt.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_dt.append(
        y_test1[i] - y_test_pred1_dt[i]
    )

Calculate performance metrics (MSE, RMSE, MAE).

In [38]:
mse_train1_dt = []
mse_test1_dt = []
rmse_train1_dt = []
rmse_test1_dt = []
mae_train1_dt = []
mae_test1_dt = []

for i, model_i in enumerate(model1_dt):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_dt[i])**2) / len(train_resid_1_dt[i])
    mse_train1_dt.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_dt[i])**2) / len(test_resid_1_dt[i])
    mse_test1_dt.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_dt.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_dt.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_dt[i])) / len(train_resid_1_dt[i])
    mae_train1_dt.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_dt[i])) / len(test_resid_1_dt[i])
    mae_test1_dt.append(mae_test_i)

In [39]:
# Organize metrics into dataframe
alg1_results_dt = pd.DataFrame(
    [mse_train1_dt, mse_test1_dt, rmse_train1_dt, rmse_test1_dt, mae_train1_dt, mae_test1_dt],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_dt

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,10.270995,13.008020,3.204839,3.606663,2.508087,2.816019
M2,10.199319,13.083917,3.193637,3.617170,2.497943,2.829215
M3,10.182915,13.178831,3.191068,3.630266,2.498647,2.827725
M4,10.155433,13.072026,3.186759,3.615526,2.499032,2.827862
M5,10.012303,12.988260,3.164222,3.603923,2.479194,2.821901


In [40]:
alg1_results_dt_avgs = pd.DataFrame(alg1_results_dt.mean(axis=0), columns=['DecisionTree_Avg']).T
alg1_results_dt_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
DecisionTree_Avg,10.164193,13.066211,3.188105,3.614709,2.496581,2.824544


## Random Forest

Train the model (one for each imputation)

In [41]:
# Initialize
model1_rf = np.repeat(RandomForestRegressor(max_depth=10, random_state=42), 5).tolist() # The best tree depth seems to be 10

# Train
for i, model_i in enumerate(model1_rf):
    model_i.fit(X_train1[i], y_train1[i])

In [42]:
# Generate test predictions and calculate residuals
y_test_pred1_rf = []
y_train_pred1_rf = []
test_resid_1_rf = []
train_resid_1_rf = []

for i, model_i in enumerate(model1_rf):
    # Train
    y_train_pred1_rf.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_rf.append(
        y_train1[i] - y_train_pred1_rf[i]
    )

    # Test
    y_test_pred1_rf.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_rf.append(
        y_test1[i] - y_test_pred1_rf[i]
    )


Calculate performance metrics (MSE, RMSE, MAE).

In [43]:
mse_train1_rf = []
mse_test1_rf = []
rmse_train1_rf = []
rmse_test1_rf = []
mae_train1_rf = []
mae_test1_rf = []

for i, model_i in enumerate(model1_rf):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_rf[i])**2) / len(train_resid_1_rf[i])
    mse_train1_rf.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_rf[i])**2) / len(test_resid_1_rf[i])
    mse_test1_rf.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_rf.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_rf.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_rf[i])) / len(train_resid_1_rf[i])
    mae_train1_rf.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_rf[i])) / len(test_resid_1_rf[i])
    mae_test1_rf.append(mae_test_i)


In [44]:
# Organize metrics into dataframe
alg1_results_rf = pd.DataFrame(
    [mse_train1_rf, mse_test1_rf, rmse_train1_rf, rmse_test1_rf, mae_train1_rf, mae_test1_rf],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_rf

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,8.333514,11.146509,2.886783,3.338639,2.279084,2.615595
M2,8.289016,11.214065,2.879065,3.348741,2.270723,2.628496
M3,8.273919,11.267755,2.876442,3.356748,2.271511,2.624196
M4,8.244680,11.203582,2.871355,3.347175,2.271653,2.625073
M5,8.026357,11.138111,2.833083,3.337381,2.244044,2.624825


In [45]:
alg1_results_rf_avgs = pd.DataFrame(alg1_results_rf.mean(axis=0), columns=['RandomForest_Avg']).T
alg1_results_rf_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
RandomForest_Avg,8.233497,11.194004,2.869345,3.345737,2.267403,2.623637


## K-Nearest Neighbors Regression

Train the model (one for each imputation)

In [46]:
# Initialize
model1_knn = np.repeat(KNeighborsRegressor(), 5).tolist()

# Train
for i, model_i in enumerate(model1_knn):
    model_i.fit(X_train1[i], y_train1[i])

In [47]:
# Generate test predictions and calculate residuals
y_test_pred1_knn = []
y_train_pred1_knn = []
test_resid_1_knn = []
train_resid_1_knn = []

for i, model_i in enumerate(model1_knn):
    # Train
    y_train_pred1_knn.append(
        model_i.predict(X_train1[i])
    )
    train_resid_1_knn.append(
        y_train1[i] - y_train_pred1_knn[i]
    )


    # Test
    y_test_pred1_knn.append(
        model_i.predict(X_test1[i])
    )
    test_resid_1_knn.append(
        y_test1[i] - y_test_pred1_knn[i]
    )

Calculate performance metrics (MSE, RMSE, MAE).

In [48]:
mse_train1_knn = []
mse_test1_knn = []
rmse_train1_knn = []
rmse_test1_knn = []
mae_train1_knn = []
mae_test1_knn = []

for i, model_i in enumerate(model1_knn):
    # MSE for train data
    mse_train_i = np.sum((train_resid_1_knn[i])**2) / len(train_resid_1_knn[i])
    mse_train1_knn.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_1_knn[i])**2) / len(test_resid_1_knn[i])
    mse_test1_knn.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train1_knn.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test1_knn.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_1_knn[i])) / len(train_resid_1_knn[i])
    mae_train1_knn.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_1_knn[i])) / len(test_resid_1_knn[i])
    mae_test1_knn.append(mae_test_i)

In [49]:
# Organize metrics into dataframe
alg1_results_knn = pd.DataFrame(
    [mse_train1_knn, mse_test1_knn, rmse_train1_knn, rmse_test1_knn, mae_train1_knn, mae_test1_knn],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_knn

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,18.266788,26.650395,4.273966,5.162402,3.270044,3.962679
M2,18.239861,26.823971,4.270815,5.179186,3.266937,3.977344
M3,18.227403,26.860564,4.269356,5.182718,3.260268,3.983794
M4,18.112248,26.646730,4.255849,5.162047,3.252385,3.965698
M5,17.850157,26.668240,4.224945,5.164130,3.229549,3.969681


In [50]:
alg1_results_knn_avgs = pd.DataFrame(alg1_results_knn.mean(axis=0), columns=['KNN_Avg']).T
alg1_results_knn_avgs

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
KNN_Avg,18.139291,26.72998,4.258986,5.170097,3.255837,3.971839


## Evaluate all metrics

In [51]:
pd.concat([
    bl1_results_avgs,
    bl2_results_avgs,
    alg1_results_lm_avgs,
    alg1_results_ridge_avgs,
    alg1_results_ridge_avgs_CV,
    alg1_results_lasso_avgs,
    alg1_results_dt_avgs,
    alg1_results_rf_avgs,
    alg1_results_knn_avgs
])

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
BL1,40.317699,39.834691,6.349620,6.311461,5.080345,5.054185
BL2,74.573814,74.763256,8.635612,8.646567,6.619678,6.679264
LM,17.126662,17.068605,4.138434,4.131404,3.240077,3.251967
Ridge_Avg,17.126673,17.068430,4.138435,4.131383,3.240130,3.252016
Ridge_CV_Avg,17.126665,17.068430,4.138434,4.131383,3.240093,3.252016
Lasso_Avg,21.947617,21.551439,4.684824,4.642342,3.781539,3.764863
DecisionTree_Avg,10.164193,13.066211,3.188105,3.614709,2.496581,2.824544
RandomForest_Avg,8.233497,11.194004,2.869345,3.345737,2.267403,2.623637
KNN_Avg,18.139291,26.729980,4.258986,5.170097,3.255837,3.971839


The random forest algorithm clearly has the best performance. We will use this algorithm framework moving forward. To train the models, run `~/src/models/train_model.py`. This script will export the models to `~/models/`.

## Random Forest with all imputations consolidated into one dataset

In [52]:
X_train = pd.concat(X_train1)
X_test = pd.concat(X_test1)
y_train = pd.concat(y_train1)
y_test = pd.concat(y_test1)

model = RandomForestRegressor(max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Generate test predictions and calculate residuals
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_resid = y_train - y_train_pred
test_resid = y_test - y_test_pred

mse_train = np.sum(train_resid**2) / len(train_resid)
mse_test = np.sum(test_resid**2) / len(test_resid)

rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

mae_train = np.sum(np.abs(train_resid)) / len(train_resid)
mae_test = np.sum(np.abs(test_resid)) / len(test_resid)

alg_results_cons = pd.DataFrame(
    [mse_train, mse_test, rmse_train, rmse_test, mae_train, mae_test],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['Cons_RF']
).T

alg_results_cons



,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
Cons_RF,7.889622,11.489132,2.808847,3.389562,2.207516,2.652819


## Random Forest where the average prediction for each imputation is evaluated

In [53]:
# Calculate averages of predictions across the five imputation sets
y_train_pred_avg = np.mean(y_train_pred1_rf, axis=0)
y_test_pred_avg = np.mean(y_test_pred1_rf, axis=0)

# Evaluate residuals
train_resid_avg = []
test_resid_avg = []
for i, model_i in enumerate(model1_rf):
    train_resid_avg.append(
        y_train1[i] - y_train_pred_avg
    )
    test_resid_avg.append(
        y_test1[i] - y_test_pred_avg
    )

# Calculate performance metrics
mse_train_avg = []
mse_test_avg = []
rmse_train_avg = []
rmse_test_avg = []
mae_train_avg = []
mae_test_avg = []

for i, model_i in enumerate(model1_rf):
    # MSE for train data
    mse_train_i = np.sum((train_resid_avg[i])**2) / len(train_resid_avg[i])
    mse_train_avg.append(mse_train_i)

    # MSE for test data
    mse_test_i = np.sum((test_resid_avg[i])**2) / len(test_resid_avg[i])
    mse_test_avg.append(mse_test_i)

    # RMSE for train data
    rmse_train_i = np.sqrt(mse_train_i)
    rmse_train_avg.append(rmse_train_i)

    # RMSE for test data
    rmse_test_i = np.sqrt(mse_test_i)
    rmse_test_avg.append(rmse_test_i)

    # MAE for train data
    mae_train_i = np.sum(np.abs(train_resid_avg[i])) / len(train_resid_avg[i])
    mae_train_avg.append(mae_train_i)

    # MAE for test data
    mae_test_i = np.sum(np.abs(test_resid_avg[i])) / len(test_resid_avg[i])
    mae_test_avg.append(mae_test_i)


# Organize metrics into dataframe
alg1_results_avg = pd.DataFrame(
    [mse_train_avg, mse_test_avg, rmse_train_avg, rmse_test_avg, mae_train_avg, mae_test_avg],
    index=['MSE_train', 'MSE_test', 'RMSE_train', 'RMSE_test', 'MAE_train', 'MAE_test'],
    columns=['M1', 'M2', 'M3', 'M4', 'M5']
).T

alg1_results_avg

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,8.333514,11.146509,2.886783,3.338639,2.279084,2.615595
M2,8.289016,11.214065,2.879065,3.348741,2.270723,2.628496
M3,8.273919,11.267755,2.876442,3.356748,2.271511,2.624196
M4,8.244680,11.203582,2.871355,3.347175,2.271653,2.625073
M5,8.026357,11.138111,2.833083,3.337381,2.244044,2.624825


Check the difference between these performance metrics (the average of the five predictions against each imputation's test set) and those of the original five model predictions against their own imputation's test set.

In [54]:
alg1_results_rf - alg1_results_avg

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
M1,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
M2,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
M3,1.776357e-15,0.0,4.440892e-16,0.0,-4.440892e-16,4.440892e-16
M4,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00
M5,0.000000e+00,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00


In [55]:
alg1_results_avg_avgs = pd.DataFrame(alg1_results_avg.mean(axis=0), columns=['RandomForest_Avg_Avgs']).T
pd.concat([alg1_results_rf_avgs, alg1_results_avg_avgs])

,MSE_train,MSE_test,RMSE_train,RMSE_test,MAE_train,MAE_test
RandomForest_Avg,8.233497,11.194004,2.869345,3.345737,2.267403,2.623637
RandomForest_Avg_Avgs,8.233497,11.194004,2.869345,3.345737,2.267403,2.623637


It appears that the difference in predictions between the models for each imputation is marginal. This would suggest that it doesn't matter which algorithm we use. But to be safe and thorough, we will take the average prediction between all the models.

# Algorithm 2: Incorporating Smoking and/or Pregnancy

We may examine incorporation of smoking and/or pregnancy data in future work, but for now we are settling with the features used in Algorithm 1.

# Test importing the models

In [103]:
with open('./rf_models/model1.pkl', 'rb') as f:
    model = pickle.load(f)

In [104]:
model.predict(np.array([[140, 72 * 2.54, 1, 25, 30 * 2.54]]))

array([35.14233464])